In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import signal
from scipy import linalg
import scipy.sparse as sp
import healpy as hp
import pickle

plt.rc("savefig", dpi=300)

# Common variables
nside = 16
npix = hp.nside2npix(nside)
tol = 1e-6

In [ ]:
# Normalised Mean Squared Error (NMSE)
def NMSE (arr, map_inj):
    return ((arr-map_inj).dot(arr-map_inj)/(map_inj.dot(map_inj)))

# Normalised Scalar Product
def NSP(source_map, clean_map):
    return (source_map.dot(clean_map)/np.sqrt(source_map.dot(source_map)*clean_map.dot(clean_map)))

In [ ]:
# Beam matrix and Hessian aka C
C = np.loadtxt('Hess'+str(npix)+'.txt')
B = np.loadtxt('FBeamNew_'+str(npix)+'.txt')

#print (B[100,200],B[200,100],B[100,100],B[200,200])

# Hessian for norm regularisation
Cn = np.identity(npix)

In [ ]:
figdir='fig/'
saveFigs = True

import pylab
import matplotlib.colors

# Histogram equalised colormap
# From https://stackoverflow.com/questions/5858902/histogram-equalization-of-matplotlib-color-tables
imvals = np.sort(B.flatten())
#vals = (imvals - np.min(imvals))/np.max(imvals)*256.0
#imvals = vals.astype(int)
lo = imvals[0]
hi = imvals[-1]
steps = (imvals[::int(len(imvals)/16)] - lo) / (hi - lo)
num_steps = float(len(steps))
interps = [(s, idx/num_steps, idx/num_steps) for idx, s in enumerate(steps)]
interps.append((1, 1, 1))
cdict = {'red' : interps,
         'green' : interps,
         'blue' : interps}
histeq_cmap = matplotlib.colors.LinearSegmentedColormap('HistEq', cdict)

plt.figure()
plt.imshow(B,interpolation="none",cmap=histeq_cmap)
plt.colorbar()
plt.axis([0,npix,0,npix])
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'beamMatrix.png',bbox_inches = 'tight',pad_inches = 0)

plt.figure()
plt.imshow(C)
plt.colorbar()
plt.axis([0,npix,0,npix])
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'Hessian.png',bbox_inches = 'tight',pad_inches = 0)

In [ ]:
# Noise

noiseScale = 0.25

## Covariance Matrix
#N = (0.25) * B
#
## Generate noise maps (this needs to be run only once to generate the noise files)
#nsim = 1000
#for i in range(nsim):
#    np.random.seed(i)
#    noise = np.random.multivariate_normal(np.zeros(npix),N)
#    np.savetxt('noise/noiseMap_'+str(npix)+'_'+str(i)+'.txt',noise)

noise0 = noiseScale * np.loadtxt('noise/noiseMap_'+str(npix)+'_117.txt')
hp.mollview(noise0)
if saveFigs:
    plt.savefig(figdir+'noise.pdf')

In [ ]:
# Point source
inj_pix = 736
#inj_pix = [799, 800, 801]
x = 0.05 # variable_source strength (0.5, 0.1, 0.05)

# Injection map
inj_map = np.zeros(npix)
inj_map[inj_pix] = x
hp.mollview(inj_map, title='Source Map')
if saveFigs:
    plt.savefig(figdir+'source-point'+str(x)+'.pdf')

# Dirty Map
dirty_map = B.dot(inj_map) + noise0
hp.mollview(dirty_map, title='Dirty Map')
if saveFigs:
    plt.savefig(figdir+'dirty-point'+str(x)+'.pdf')

nmse_dirty = NMSE (dirty_map, B.dot(inj_map))
nsp_dirty = NSP (dirty_map, B.dot(inj_map))

In [ ]:
# Point source: Plot NMSE and NSP vs number of iterations to find the optimal number of iteration
niter = 20
nmse_noreg_vs_iter = np.zeros(niter)
nsp_noreg_vs_iter = np.zeros(niter)
for iiter in range(niter):
    clean_map = sp.linalg.cgs(B, dirty_map, tol = tol, maxiter = iiter+1)
    nmse_noreg_vs_iter[iiter] = NMSE (clean_map[0],inj_map)
    nsp_noreg_vs_iter[iiter] = NSP (clean_map[0],inj_map)

plt.figure()
plt.plot(nmse_noreg_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NMSE')
plt.title('Point Source: No regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nmse_no-reg_vs_iter_point'+str(x)+'.pdf')
plt.show()

plt.figure()
plt.plot(nsp_noreg_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NSP')
plt.title('Point Source: No regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nsp_no-reg_vs_iter_point'+str(x)+'.pdf')
plt.show()

# Optimum number of iterations
opt_iters_noreg = np.argmin(nmse_noreg_vs_iter) + 1
print ("Optimum number of iterations: ", opt_iters_noreg)

In [ ]:
# Point Source: Unregularised deconvolution
#iters_noreg = opt_iters_noreg
iters_noreg = 3

# Clean Map: DECONVOLUTION WITHOUT REGULARIZATION:
clean_map = sp.linalg.cgs(B, dirty_map, tol = tol, maxiter = iters_noreg)
hp.mollview(clean_map[0], title='Clean Map: Unregularised, #iterations = '+str(iters_noreg))
if saveFigs:
    plt.savefig(figdir+'clean_no-reg_point'+str(x)+'_iters'+str(iters_noreg)+'.pdf')
nmse_noreg = NMSE (clean_map[0],inj_map)
nsp_noreg = NSP (clean_map[0],inj_map)
print (nmse_noreg,nsp_noreg)

In [ ]:
# Point Source: Plot NMSE and NSP vs lambda to find the optimal regularisation strength
maxiter = 50

maxlamb = 1e6
minlamb = 1
nlambda = 100
alambda = np.logspace(np.log10(minlamb), np.log10(maxlamb), nlambda, endpoint=True)
#alambda = np.linspace(minlamb, maxlamb, nlambda, endpoint=True)

nmse_norm_vs_lambda = np.zeros(nlambda)
nsp_norm_vs_lambda = np.zeros(nlambda)
for ilambda in range(nlambda):
    clean_map_norm = sp.linalg.cgs((B+alambda[ilambda]*Cn),dirty_map,tol = tol, maxiter = maxiter)
    nmse_norm_vs_lambda[ilambda] = NMSE (clean_map_norm[0],inj_map)
    nsp_norm_vs_lambda[ilambda]  = NSP (clean_map_norm[0],inj_map)

plt.figure()
plt.semilogx(alambda,nmse_norm_vs_lambda)
plt.xlabel('Regularisation strength')
plt.ylabel('NMSE')
plt.title('Point Source: Norm Regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nmse_norm_vs_lambda_point'+str(x)+'.pdf')
plt.show()

plt.figure()
plt.semilogx(alambda,nsp_norm_vs_lambda)
plt.xlabel('Regularisation strength')
plt.ylabel('NSP')
plt.title('Point Source: Norm Regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nsp_norm_vs_lambda_point'+str(x)+'.pdf')
plt.show()

# Optimal lambda
optlamb_point = alambda[np.argmin(nmse_norm_vs_lambda)]
print ("Optimum regularisation strength for point source with norm regularisation: ", optlamb_point)

In [ ]:
# Point source: Regularisation strength
#lamb = optlamb_point
lamb = 1000

# Clean Map: DECONVOLUTION With Norm Regularisation:
clean_map_norm = sp.linalg.cgs(B+lamb*Cn, dirty_map, tol = tol, maxiter = maxiter)
hp.mollview(clean_map_norm[0], title='Clean Map: Norm regularisation with strength '+str(lamb))
if saveFigs:
    plt.savefig(figdir+'clean_norm-reg_point'+str(x)+'_lambda'+str(lamb)+'.pdf')
nmse_norm = NMSE (clean_map_norm[0],inj_map)
nsp_norm = NSP (clean_map_norm[0],inj_map)

# DECONVOLUTION With Grad Regularisation:
clean_map_grad = sp.linalg.cgs(B+lamb*C, dirty_map, tol = tol, maxiter = maxiter)
hp.mollview(clean_map_grad[0], title='Clean Map: Gradient regularisation with strength = '+str(lamb))
if saveFigs:
    plt.savefig(figdir+'clean_grad-reg_point'+str(x)+'_lambda'+str(lamb)+'.pdf')
nmse_grad = NMSE (clean_map_grad[0],inj_map)
nsp_grad = NSP (clean_map_grad[0],inj_map)

print (x,'&', iters_noreg,'&',lamb,'&', \
       "%7.4F" %(nsp_dirty),'&', "%7.4F" %(nsp_noreg),'&', "%7.4F" %(nsp_norm),'&', "%7.4F" %(nsp_grad),'&',\
       "%9.4F" %(nmse_dirty),'&', "%9.4F" %(nmse_noreg),'&', "%9.4F" %(nmse_norm),'&', "%9.4F" %(nmse_grad), '\\\hline')

In [ ]:
# Point Source: Masked maps

# Dirty map
dirty_map_sigma = np.std(dirty_map)

dirty_map_2s = dirty_map.copy()
dirty_map_2s[dirty_map_2s < 2.0*dirty_map_sigma] = 0.0
hp.mollview(dirty_map_2s, title='Dirty Map: 2-sigma mask')
if saveFigs:
    plt.savefig(figdir+'dirty-point'+str(x)+'_2sigma-mask.pdf')
dirty_map_3s = dirty_map.copy()
dirty_map_3s[dirty_map_3s < 3.0*dirty_map_sigma] = 0.0
hp.mollview(dirty_map_3s, title='Dirty Map: 3-sigma mask')
if saveFigs:
    plt.savefig(figdir+'dirty-point'+str(x)+'_3sigma-mask.pdf')

# Unregularised Clean Map
clean_map_sigma = np.std(clean_map[0])
clean_map_2s = clean_map[0].copy()
clean_map_2s[clean_map_2s < 2.0*clean_map_sigma] = 0.0
hp.mollview(clean_map_2s, title='Unregularised Clean Map: 2-sigma mask')
if saveFigs:
    plt.savefig(figdir+'clean_no-reg_point'+str(x)+'_iters'+str(iters_noreg)+'_2sigma-mask.pdf')
clean_map_3s = clean_map[0].copy()
clean_map_3s[clean_map_3s < 3.0*clean_map_sigma] = 0.0
hp.mollview(clean_map_3s, title='Unregularised Clean Map: 3-sigma mask')
if saveFigs:
    plt.savefig(figdir+'clean_no-reg_point'+str(x)+'_iters'+str(iters_noreg)+'_3sigma-mask.pdf')

# Norm regularised Clean Map
clean_map_norm_sigma = np.std(clean_map_norm[0])
clean_map_norm_2s = clean_map_norm[0].copy()
clean_map_norm_2s[clean_map_norm_2s < 2.0*clean_map_norm_sigma] = 0.0
hp.mollview(clean_map_norm_2s, title='Norm regularised Clean Map: 2-sigma mask')
if saveFigs:
    plt.savefig(figdir+'clean_norm-reg_point'+str(x)+'_lambda'+str(lamb)+'_2sigma-mask.pdf')
clean_map_norm_3s = clean_map_norm[0].copy()
clean_map_norm_3s[clean_map_norm_3s < 3.0*clean_map_norm_sigma] = 0.0
hp.mollview(clean_map_norm_3s, title='Norm regularised Clean Map: 3-sigma mask')
if saveFigs:
    plt.savefig(figdir+'clean_norm-reg_point'+str(x)+'_lambda'+str(lamb)+'_3sigma-mask.pdf')

In [ ]:
# Point Source: Stability of Deconvolution
niter = maxiter
nmse_norm_vs_iter = np.zeros(niter)
nsp_norm_vs_iter = np.zeros(niter)
for iiter in range(niter):
    clean_map_norm = sp.linalg.cgs(B+lamb*Cn, dirty_map, tol = tol, maxiter = iiter+1)
    nmse_norm_vs_iter[iiter] = NMSE (clean_map_norm[0],inj_map)
    nsp_norm_vs_iter[iiter] = NSP (clean_map_norm[0],inj_map)

plt.figure()
plt.plot(nmse_norm_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NMSE')
plt.title('Point Source: Norm Regularisation with strength '+str(lamb))
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nmse_norm_vs_iter_point'+str(x)+'_lambda'+str(lamb)+'.pdf')
plt.show()

plt.figure()
plt.plot(nsp_norm_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NSP')
plt.title('Point Source: Norm Regularisation with strength '+str(lamb))
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nsp_norm_vs_iter_point'+str(x)+'_lambda'+str(lamb)+'.pdf')
plt.show()

In [ ]:
# Point Source: Simulations
#iters_noreg = 5
#lamb = 1000
nsim = 1000   # NUMBER OF ELEMENTS
strengthRange = [0.05,0.1]
sourceStrength = np.zeros(nsim)
injPix = np.zeros(nsim)
nmse_norm = np.zeros(nsim)
nmse_unreg = np.zeros(nsim)
inj_map = np.zeros(npix)
for i in range(nsim):
    noise = noiseScale * np.loadtxt('noise/noiseMap_'+str(npix)+'_'+str(i+1)+'.txt')
    np.random.seed(i)
    sourceStrength[i] = random.uniform(strengthRange[0],strengthRange[1])
    randomPix = random.randint(0,npix-1)
    injPix[i] = randomPix
    inj_map[:] = 0.0
    inj_map[randomPix] = sourceStrength[i]
    dirty_map = B.dot(inj_map) + noise

    # DECONVOLUTION WITHOUT REGULARIZATION:
    clean_map = sp.linalg.cgs(B, dirty_map, tol = tol, maxiter = iters_noreg)
    nmse_unreg[i] = NMSE(clean_map[0],inj_map)  
    # DECONVOLUTION WITH REGULARIZATION:
    clean_map_norm = sp.linalg.cgs(B+lamb*Cn, dirty_map, tol = tol, maxiter = maxiter)
    nmse_norm[i] = NMSE(clean_map_norm[0],inj_map)
    
    if np.remainder((i+1),100) == 0:
        print ("%6d" %(i+1),' of ', "%6d" %(nsim),' simulations done')

print(min(nmse_unreg - nmse_norm)) # Positive value will mean regularisation is always better

np.savetxt('point_norm_sims_lambda'+str(lamb)+'.txt',\
           np.column_stack((injPix, sourceStrength, nmse_unreg, nmse_norm)))

n, bins, patches = plt.hist((nmse_norm, nmse_unreg, nmse_unreg - nmse_norm), bins=20,\
                            label=['Norm regularisation','No regularisation','Without - With Regularisation'],\
                            color=['tab:green','tab:orange','tab:blue'])
hatches = ['//', '--', '']
for patch_set, hatch in zip(patches, hatches):
    plt.setp(patch_set, hatch=hatch)
plt.legend()
plt.xlabel('NMSE')
plt.title('Histogram for point sources: Regularisation strength '+str(lamb))
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'hist_point_norm_lambda'+str(lamb)+'.pdf')
plt.show()

In [ ]:
# Extended source
y = 0.01 # Source strength (0.1, 0.025, 0.01)

sourceMap = np.loadtxt('FSource_galaxy.txt')
inj_map_ext = (y / max(sourceMap)) * sourceMap
hp.mollview(inj_map_ext, title='Source Map')
if saveFigs:
    plt.savefig(figdir+'source-ext'+str(y)+'.pdf')

dirty_map_ext = B.dot(inj_map_ext) + noise0
hp.mollview(dirty_map_ext, title='Dirty Map')
if saveFigs:
    plt.savefig(figdir+'dirty-ext'+str(y)+'.pdf')

nmse_dirty = NMSE (dirty_map_ext, B.dot(inj_map_ext))
nsp_dirty = NSP (dirty_map_ext, B.dot(inj_map_ext))

In [ ]:
# Extended source: Plot NMSE and NSP vs number of iterations to find the optimal number of iterations
niter = 100
nmse_noreg_vs_iter = np.zeros(niter)
nsp_noreg_vs_iter = np.zeros(niter)
for iiter in range(niter):
    clean_map_ext = sp.linalg.cgs(B, dirty_map_ext, tol = tol, maxiter = iiter+1)
    nmse_noreg_vs_iter[iiter] = NMSE (clean_map_ext[0],inj_map_ext)
    nsp_noreg_vs_iter[iiter]  = NSP (clean_map_ext[0],inj_map_ext)

plt.figure()
plt.plot(nmse_noreg_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NMSE')
plt.title('Extended Source: No regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nmse_no-reg_vs_iter_ext'+str(y)+'.pdf')
plt.show()

plt.figure()
plt.plot(nsp_noreg_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NSP')
plt.title('Extended Source: No regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nsp_no-reg_vs_iter_ext'+str(y)+'.pdf')
plt.show()

# Optimum number of iterations
opt_iters_noreg_ext = np.argmax(nsp_noreg_vs_iter) + 1
print ("Optimum number of iterations: ", opt_iters_noreg_ext)

In [ ]:
# Extended Source: Unregularised deconvolution
#iters_noreg = opt_iters_noreg_ext
iters_noreg = 3

# Clean Map: DECONVOLUTION WITHOUT REGULARIZATION:
clean_map_ext = sp.linalg.cgs(B, dirty_map_ext, tol = tol, maxiter = iters_noreg)
hp.mollview(clean_map_ext[0], title='Clean Map: Unregularised, #iterations = '+str(iters_noreg))
if saveFigs:
    plt.savefig(figdir+'clean_no-reg_ext'+str(y)+'_iters'+str(iters_noreg)+'.pdf')
nmse_noreg = NMSE (clean_map_ext[0],inj_map_ext)
nsp_noreg = NSP (clean_map_ext[0],inj_map_ext)
print (nmse_noreg,nsp_noreg)

In [ ]:
# Extended Source: Plot NMSE and NSP vs lambda to find the optimal regularisation strength
maxiter = 100

maxlamb = 1e6
minlamb = 1
nlambda = 100
alambda = np.logspace(np.log10(minlamb), np.log10(maxlamb), nlambda, endpoint=True)
#alambda = np.linspace(minlamb, maxlamb, nlambda, endpoint=True)

nmse_grad_vs_lambda = np.zeros(nlambda)
nsp_grad_vs_lambda = np.zeros(nlambda)
for ilambda in range(nlambda):
    clean_map_grad_ext = sp.linalg.cgs((B+alambda[ilambda]*C),dirty_map_ext,tol = tol, maxiter = maxiter)
    nmse_grad_vs_lambda[ilambda] = NMSE (clean_map_grad_ext[0],inj_map_ext)
    nsp_grad_vs_lambda[ilambda]  = NSP (clean_map_grad_ext[0],inj_map_ext)

plt.figure()
plt.semilogx(alambda,nmse_grad_vs_lambda)
plt.xlabel('Regularisation strength')
plt.ylabel('NMSE')
plt.title('Extended Source: Gradient Regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nmse_grad_vs_lambda_ext'+str(y)+'.pdf')
plt.show()

plt.figure()
plt.semilogx(alambda,nsp_grad_vs_lambda)
plt.xlabel('Regularisation strength')
plt.ylabel('NSP')
plt.title('Extended Source: Gradient Regularisation')
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nsp_grad_vs_lambda_ext'+str(y)+'.pdf')
plt.show()

# Optimal lambda
optlamb_ext = alambda[np.argmax(nsp_grad_vs_lambda)]
print ("Optimum regularisation strength for extended source with grad regularisation: ", optlamb_ext)

In [ ]:
# Extended source: Regularisation strength
#lamb = optlamb_ext
lamb = 500

# Clean Map: DECONVOLUTION With Norm Regularisation:
clean_map_norm_ext = sp.linalg.cgs((B+lamb*Cn),dirty_map_ext,tol = tol, maxiter = maxiter)
hp.mollview(clean_map_norm_ext[0], title='Clean Map: Norm regularisation with strength = '+str(lamb))
if saveFigs:
    plt.savefig(figdir+'clean_norm-reg_ext'+str(y)+'_lambda'+str(lamb)+'.pdf')
nmse_norm = NMSE (clean_map_norm_ext[0],inj_map_ext)
nsp_norm = NSP (clean_map_norm_ext[0],inj_map_ext)

# Clean Map: DECONVOLUTION With gradient Regularisation:
clean_map_grad_ext = sp.linalg.cgs((B+lamb*C),dirty_map_ext,tol = tol, maxiter = maxiter)
hp.mollview(clean_map_grad_ext[0], title='Clean Map: Gradient regularisation with strength = '+str(lamb))
if saveFigs:
    plt.savefig(figdir+'clean_grad-reg_ext'+str(y)+'_lambda'+str(lamb)+'.pdf')
nmse_grad = NMSE (clean_map_grad_ext[0],inj_map_ext)
nsp_grad = NSP (clean_map_grad_ext[0],inj_map_ext)

print (y,'&', iters_noreg,'&', lamb,'&',\
       "%7.4F" %(nsp_dirty),'&', "%7.4F" %(nsp_noreg),'&', "%7.4F" %(nsp_norm),'&', "%7.4F" %(nsp_grad),'&',\
       "%9.4F" %(nmse_dirty),'&', "%9.4F" %(nmse_noreg),'&', "%9.4F" %(nmse_norm),'&', "%9.4F" %(nmse_grad), '\\\hline')

In [ ]:
# Extended Source: Stability of Deconvolution
niter = maxiter
nmse_grad_vs_iter = np.zeros(niter)
nsp_grad_vs_iter = np.zeros(niter)
for iiter in range(niter):
    clean_map_grad_ext = sp.linalg.cgs(B+lamb*C, dirty_map_ext, tol = tol, maxiter = iiter+1)
    nmse_grad_vs_iter[iiter] = NMSE (clean_map_grad_ext[0],inj_map_ext)
    nsp_grad_vs_iter[iiter] = NSP (clean_map_grad_ext[0],inj_map_ext)

plt.figure()
plt.plot(nmse_grad_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NMSE')
plt.title('Extended Source: Gradient Regularisation with strength '+str(lamb))
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nmse_grad_vs_iter_ext'+str(x)+'_lambda'+str(lamb)+'.pdf')
plt.show()

plt.figure()
plt.plot(nsp_grad_vs_iter)
plt.xlabel('number of iterations')
plt.ylabel('NSP')
plt.title('Extended Source: Gradient Regularisation with strength '+str(lamb))
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'nsp_grad_vs_iter_ext'+str(y)+'_lambda'+str(lamb)+'.pdf')
plt.show()

In [ ]:
# Extended Source: Simulations
nsim = 1000   # NUMBER OF ELEMENTS
#iters_noreg = 50
#lamb = 1000
#lmax = 32
strengthRange = [0.01,0.05]
sourceStrength = np.zeros(nsim)
nsp_grad = np.zeros(nsim)
nsp_unreg = np.zeros(nsim)    

sourceMap0 = np.loadtxt('FSource_galaxy.txt')
cl0 = hp.anafast(sourceMap0)

for i in range(nsim):
    noise = noiseScale * np.loadtxt('noise/noiseMap_'+str(npix)+'_'+str(i+1)+'.txt')
    np.random.seed(i)
    sourceStrength[i] = random.uniform(strengthRange[0],strengthRange[1])
    sourceMap = np.abs(hp.synfast(cl0, nside=nside, verbose=False))
    inj_map_ext = (sourceStrength[i]/max(sourceMap)) * sourceMap
    dirty_map_ext = B.dot(inj_map_ext) + noise
    
    clean_map_ext = sp.linalg.cgs(B, dirty_map_ext, tol = tol, maxiter = iters_noreg)
    nsp_unreg[i] = NSP(clean_map_ext[0],inj_map_ext)
    
    clean_map_grad_ext = sp.linalg.cgs(B + lamb*C, dirty_map_ext, tol = tol, maxiter = maxiter)
    nsp_grad[i] = NSP(clean_map_grad_ext[0],inj_map_ext)
    
    if np.remainder((i+1),100) == 0:
        print ("%6d" %(i+1),' of ', "%6d" %(nsim),' simulations done')

print(min(nsp_grad - nsp_unreg)) # Positive value will mean regularisation is always better
    
np.savetxt('ext_grad_sims_lambda'+str(lamb)+'.txt',\
           np.column_stack((sourceStrength, nsp_unreg, nsp_grad)))

n, bins, patches = plt.hist((nsp_grad, nsp_unreg, nsp_grad - nsp_unreg), bins=20,\
                            label=['Gradient regularisation', 'No regularisation', 'With - Without Regularisation'],\
                            color=['tab:green','tab:orange','tab:blue'])

hatches = ['//', '--', '']
for patch_set, hatch in zip(patches, hatches):
    plt.setp(patch_set, hatch=hatch)
plt.legend()
plt.xlabel('NSP')
plt.title('Histogram for extended sources: Regularisation strength '+str(lamb))
plt.tight_layout()
if saveFigs:
    plt.savefig(figdir+'hist_ext_grad_lambda'+str(lamb)+'.pdf')
plt.show()